In [5]:
import sys

sys.path.append(r"..\\")

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

TORNET_DATA_INPUT_FOLDER = r"/mnt/c/users/handypark/Documents/Grad_School_Courses/CS_230/tornet"

In [6]:
"""
TorNet's data loading code, re-imported here manually for loading data into TensorFlow.
For some reason, trying to import the data loading code using `from tornet.data.tf.loader` wasn't working as expected,
so we re-copy that code over here to make use of it.
"""
from typing import List, Dict

from tornet.data.loader import query_catalog, read_file
from tornet.data.constants import ALL_VARIABLES
from tornet.data import preprocess as pp

def create_tf_dataset(files:str,
                      variables: List[str]=ALL_VARIABLES,
                      n_frames:int=1,
                      tilt_last: bool=True) -> tf.data.Dataset:
    """
    Creates a TF dataset object via the function read_file.   
    This dataset is somewhat slow because of the use of 
    tf.data.dataset.from_generator.  It is recommended to
    use this only as a means to call ds.save() to create a 
    much faster copy of the dataset.
    """
    assert len(files)>0
    # grab one file to gets keys, shapes, etc
    data = read_file(files[0],variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    
    output_signature = { k:tf.TensorSpec(shape=data[k].shape,dtype=data[k].dtype,name=k) for k in data }
    def gen():
        for f in files:
            yield read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    ds = tf.data.Dataset.from_generator(gen,
                                        output_signature=output_signature)
    return ds
    

def shard_function(data: tf.Tensor) -> np.int64:
    """
    Function that "shards" the data in tf.data.Dataset.save().
    This transforms time stamp into a np.int64 between 0,..,9.
    This is optional and may make loading faster by utilizing more CPUs.
    
    """
    x = (data['time'][0]//10) % 10 # uses tens digit of epoch time for shard index
    if x % 10 == 0:
        return np.int64(0)
    elif x % 10 == 1:
        return np.int64(1)
    elif x % 10 == 2:
        return np.int64(2)
    elif x % 10 == 3:
        return np.int64(3)
    elif x % 10 == 4:
        return np.int64(4)
    elif x % 10 == 5:
        return np.int64(5)
    elif x % 10 == 6:
        return np.int64(6)
    elif x % 10 == 7:
        return np.int64(7)
    elif x % 10 == 8:
        return np.int64(8)
    elif x % 10 == 9:
        return np.int64(9)
    else:
        return np.int64(0)



def make_tf_loader(data_root: str, 
            data_type:str='train', # or 'test'
            years: list=list(range(2013,2023)),
            batch_size: int=128, 
            weights: Dict=None,
            include_az: bool=False,
            random_state:int=1234,
            select_keys: list=None,
            tilt_last: bool=True,
            from_tfds: bool=False,
            tfds_data_version: str='1.1.0'):
    """
    Initializes tf.data Dataset for training CNN Tornet baseline.

    data_root - location of TorNet
    data_Type - 'train' or 'test'
    years     - list of years btwn 2013 - 2022 to draw data from
    batch_size - batch size
    weights - optional sample weights, see note below
    include_az - if True, coordinates also contains az field
    random_state - random seed for shuffling files
    select_keys - Only generate a subset of keys from each tornet sample
    tilt_last - If True (default), order of dimensions is left as [batch,azimuth,range,tilt]
                If False, order is permuted to [batch,tilt,azimuth,range]
    from_tfds - Use TFDS data loader, requires this version to be
                built and TFDS_DATA_ROOT to be set.  
                See tornet/data/tdfs/tornet/README.
                If False (default), the basic loader is used
    
    If you leave from_tfds as False, I suggest adding ds=ds.cache( LOCATION ) 
    in the training script to cache the dataset to speed up training times (after epoch 1)
    
    See the DataLoaders.ipynb notebook for details on how to resave TorNet in this way

    weights is optional, if provided must be a dict of the form
      weights={'wN':wN,'w0':w0,'w1':w1,'w2':w2,'wW':wW}
    where wN,w0,w1,w2,wW are numeric weights assigned to random,
    ef0, ef1, ef2+ and warnings samples, respectively.  

    After loading TorNet samples, this does the following preprocessing:
    - Optionaly permutes order of dimensions to not have tilt last
    - adds 'coordinates' variable used by CoordConv layers. If include_az is True, this
      includes r, r^{-1} (and az if include_az is True)
    - Takes only last time frame
    - Splits sample into inputs,label
    - If weights is provided, returns inputs,label,sample_weights

    """    
    if from_tfds: # fast loader
        import tensorflow_datasets as tfds
        import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'
        ds = tfds.load('tornet:%s' % tfds_data_version ,split='+'.join(['%s-%d' % (data_type,y) for y in years]))
        # Assumes data was saved with tilt_last=True and converts it to tilt_last=False
        if not tilt_last:
            ds = ds.map(lambda d: pp.permute_dims(d,(0,3,1,2), backend=tf))
    else: # Load directly from netcdf files
        file_list = query_catalog(data_root, data_type, years, random_state)
        ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1, tilt_last=tilt_last) 

    ds=preproc(ds,weights,include_az,select_keys,tilt_last)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    return ds

def preproc(ds: tf.data.Dataset,
            weights:Dict=None,
            include_az:bool=False,
            select_keys:list=None,
            tilt_last:bool=True):
    """
    Adds preprocessing steps onto dataloader
    """

    # Remove time dimesnion
    ds = ds.map(pp.remove_time_dim)

    # Add coordiante tensors
    ds = ds.map(lambda d: pp.add_coordinates(d,include_az=include_az,tilt_last=tilt_last,backend=tf))

    # split into X,y
    ds = ds.map(pp.split_x_y)

    # Add sample weights
    if weights:
        ds = ds.map(lambda x,y:  pp.compute_sample_weight(x,y,**weights, backend=tf) )
    
        # select keys for input
        if select_keys is not None:
            ds = ds.map(lambda x,y,w: (pp.select_keys(x,keys=select_keys),y,w))
    else:
        if select_keys is not None:
            ds = ds.map(lambda x,y: (pp.select_keys(x,keys=select_keys),y))

    return ds

In [7]:
def grab_data_from_given_years(years=[], type="train"):
    """
    Get the data for a given year, just to see what it looks like, and save it as a TF dataset.
    Based on TorNet guide to loading data in notebooks\ensorflow.ipynb.
    """
    split_type = type #train or test
    catalog_path = os.path.join(TORNET_DATA_INPUT_FOLDER, "catalog.csv")
            
    catalog = pd.read_csv(catalog_path, parse_dates=["start_time", "end_time"])
    catalog = catalog[catalog["type"] == split_type]
    catalog = catalog[catalog.start_time.dt.year.isin(years)]
    catalog = catalog.sample(frac=1, random_state=1234)
    file_list = [os.path.join(TORNET_DATA_INPUT_FOLDER, f) for f in catalog.filename]
    
    dataset = create_tf_dataset(file_list, variables=ALL_VARIABLES, n_frames=1)
    return dataset

In [8]:
dataset = grab_data_from_given_years([2016], "train")


I0000 00:00:1731372611.904914    4678 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5254 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
def peek_at_tf_data(dataset):
    counter = 0
    for i in dataset:
        counter += 1
        if (counter < 10):
            print(i)
        else:
            break

def save_dataset(dataset, location):
    dataset.save(location)

def load_dataset(location):
    return tf.data.Dataset.load(location)

In [10]:
peek_at_tf_data(dataset)

{'DBZ': <tf.Tensor: shape=(1, 120, 240, 2), dtype=float32, numpy=
array([[[[ nan,  nan],
         [ nan,  nan],
         [ nan,  nan],
         ...,
         [ nan,  nan],
         [ nan,  nan],
         [ nan,  nan]],

        [[ nan,  nan],
         [ nan,  nan],
         [ nan,  nan],
         ...,
         [ nan,  nan],
         [ nan,  nan],
         [ nan,  nan]],

        [[ nan,  nan],
         [ nan,  nan],
         [ nan,  nan],
         ...,
         [ nan,  nan],
         [ nan,  nan],
         [ nan,  nan]],

        ...,

        [[21.5, 22.5],
         [26.5, 28. ],
         [23. , 29. ],
         ...,
         [14.5,  9.5],
         [11.5, 11.5],
         [11.5, 16. ]],

        [[22. , 24.5],
         [22.5, 30. ],
         [24.5, 25.5],
         ...,
         [15.5, 16. ],
         [18. , 17. ],
         [20.5, 12.5]],

        [[21. , 32.5],
         [19. , 27.5],
         [25. , 27. ],
         ...,
         [20.5, 17. ],
         [18.5, 11.5],
         [20. , 13.5]

In [11]:
#save_dataset(dataset, "test.tfdataset")

In [12]:
#reloaded_dataset = load_dataset("test.tfdataset")

### How does a really simple (and terrible) CNN perform with a small shard of the data?

The answer, as one might expect, is: not so well.
But, we can verify this and make sure that our architecture is running the way we expect it to!
To do so, we'll execute the following steps (based on TorNet's repository, with some edits and refactoring that will come in handy later):
- Setup functions to preprocess the data
- Train a simple CNN model based on the 2016 train dataset which only does the following things:
    - Normalizes each input dimension
    - Concatenates the input images
    - Runs a Conv2D layer (after removing NaN values)
    - Gets an output prediction
- Test our simple CNN model with the 2018 test dataset

In [13]:
import tornet.data.preprocess as pp
from tornet.data import preprocess as tfpp

"""
Just taking the preprocessing done by TorNet (adding coordinates for CoordConv,
only using the last time frame since we only care about detecting
whether there was or wasn't a tornado at the given time,
and setting up the data as a split [inputs, label]
"""
def preprocess_data(dataset):

    # add 'coordinates' variable used by CoordConv layers
    dataset = dataset.map(lambda d: pp.add_coordinates(d, include_az=False, backend=tf))
         
    # Take only last time frame
    dataset = dataset.map(pp.remove_time_dim)
    
    # Split sample into inputs,label
    dataset = dataset.map(tfpp.split_x_y)
    
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.batch(32)
    
    return dataset

"""
Similarly, making use of TorNet's example code to create a simple CNN model.
"""
def simple_model():
    
    # Create a simple CNN model
    # This normalizes data, concatenates along channel, and applies a Conv2D
    import keras
    from tornet.data.constants import CHANNEL_MIN_MAX
    
    input_vars = ALL_VARIABLES # which variables to use
    
    # TF convention is B,L,W,H
    inputs = {v:keras.Input(shape=(120,240,2),name=v) for v in input_vars}
    
    # Normalize inputs
    norm_layers = []
    for v in input_vars:
        min_max = np.array(CHANNEL_MIN_MAX[v]) # [2,]
    
        # choose mean,var to get approximate [-1,1] scaling
        var=((min_max[1]-min_max[0])/2)**2 # scalar
        var=np.array(2*[var,])    # [n_sweeps,]
        offset=(min_max[0]+min_max[1])/2    # scalar
        offset=np.array(2*[offset,]) # [n_sweeps,]
        
        norm_layers.append(
            keras.layers.Normalization(mean=offset, variance=var,
                                       name='Normalized_%s' % v)
        )
    
    # Concatenate normed inputs along channel dimension
    x=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
            [l(inputs[v]) for l,v in zip(norm_layers,input_vars)]
            )
    
    # Replace background (nan) with -3
    x=keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x),-3.0,x),name='ReplaceNan')(x)
    
    # Processing
    x = keras.layers.Conv2D(32,3,padding='same',activation='relu')(x)
    # add more..
    x = keras.layers.Conv2D(1,1,padding='same',activation='relu',name='TornadoLikelihood')(x)
    y = keras.layers.GlobalMaxPool2D(name='GlobalMaxPool')(x)
    
    model = keras.Model(inputs=inputs,outputs=y,name='TornadoDetector')
    model.summary()
    return model

In [15]:
import keras
model = simple_model()
opt  = keras.optimizers.Adam(learning_rate=1e-3)
loss=keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

Model: "TornadoDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_DBZ      │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_VEL      │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_KDP      │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_RHOHV    │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_ZDR      │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_WIDTH    │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalized_DBZ[0… │
│ (Concatenate)       │ 12)               │            │ Normalized_VEL[0… │
│                     │                   │            │ Normalized_KDP[0… │
│                     │                   │            │ Normalized_RHOHV… │
│                     │                   │            │ Normalized_ZDR[0… │
│                     │                   │            │ Normalized_WIDTH… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ReplaceNan (Lambda) │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 120, 240,  │      3,488 │ ReplaceNan[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,521 (13.75 KB)

 Trainable params: 3,521 (13.75 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
preprocessed = preprocess_data(dataset)
model.fit(preprocessed,epochs=3,steps_per_epoch=20)

Epoch 1/3


/usr/local/lib/python3.9/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(
2024-11-11 16:50:37.623902: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 51673152 bytes after encountering the first element of size 51673152 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
I0000 00:00:1731372637.683162    5219

 2/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.7099 

I0000 00:00:1731372642.240225    5219 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 1.6656
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - loss: 0.7283
Epoch 3/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - loss: 0.6954


In [17]:
test_data = grab_data_from_given_years([2018], "test")
preprocessed_test_data = preprocess_data(test_data)

metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
model.compile(loss=loss,metrics=metrics)

model.evaluate(preprocessed_test_data, steps=10)

/usr/local/lib/python3.9/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - AUC: 0.4871 - loss: 0.6975


[0.6955778002738953, 0.506406307220459]

Results are pretty terrible, but still better than 50% AUC (which means we have barely any predictive power, but it's still an "educated" guess rather than a completely random guess.

Of note, we do see that as we increase the number of epochs, the model does seem to fit better and better.
In other words, at the very least, our training is working (even if the model we're training in this preliminary example is terrible).

Out of curiosity, how would we do if we were to use all of our data from several years, even with a "terribly shallow" NN architecture?

In [18]:
model_many = simple_model()
opt = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model_many.compile(loss=loss, optimizer=opt)

many_years_train = grab_data_from_given_years(range(2013, 2021), "train")
preprocessed_many_years = preprocess_data(many_years_train)

# we'll use Adam optimization for training and typical binary cross entropy loss for binary classification
opt = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model_many.compile(loss=loss, optimizer=opt)

model_many.fit(preprocessed_many_years, epochs=3, steps_per_epoch=20)

Model: "TornadoDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_DBZ      │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_VEL      │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_KDP      │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_RHOHV    │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_ZDR      │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_WIDTH    │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalized_DBZ[0… │
│ (Concatenate)       │ 12)               │            │ Normalized_VEL[0… │
│                     │                   │            │ Normalized_KDP[0… │
│                     │                   │            │ Normalized_RHOHV… │
│                     │                   │            │ Normalized_ZDR[0… │
│                     │                   │            │ Normalized_WIDTH… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ReplaceNan (Lambda) │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 120, 240,  │      3,488 │ ReplaceNan[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,521 (13.75 KB)

 Trainable params: 3,521 (13.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3


/usr/local/lib/python3.9/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - loss: 3.0337
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 0.8072
Epoch 3/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - loss: 0.7137


In [19]:
test_data = grab_data_from_given_years([2022], "test")
preprocessed_test_data = preprocess_data(test_data)

metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
model_many.compile(loss=loss,metrics=metrics)

model_many.evaluate(preprocessed_test_data, steps=20)

/usr/local/lib/python3.9/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - AUC: 0.4985 - loss: 0.7175


[0.7223304510116577, 0.5413710474967957]

Results are still quite terrible, in fact - they're even worse, which makes complete sense. 

Still, we've shown a few things here:
- We can run the NN training on Tensorflow as hoped, with pretty quick training times for reasonably sized neural nets.
- We do need a NN architecture that will actually learn from the data and actually have predictive power (rather than a single Conv2D layer).
- We definitely know how to overfit on a model.

In terms of preliminaries, these are pretty good first steps, because we've demonstrated that the training process is computationally tractable given the hardware being used (just an NVIDIA GeForce RTX 3070 on a gaming desktop PC with CUDA support + Tensorflow) for the scale of data that we're working with, even when using all of the data in question. 

Now it's time to actually make models with predictive power.